<a href="https://colab.research.google.com/github/l43lu/NLP_Project_Assignment/blob/master/German_News_Articles_Data_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install stopwordsiso

     |████████████████████████████████| 73 kB 2.3 MB/s 


In [ ]:
!pip install nlpaug

     |████████████████████████████████| 405 kB 13.4 MB/s 


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.9 MB 13.9 MB/s 
     |████████████████████████████████| 3.3 MB 28.0 MB/s 
     |████████████████████████████████| 56 kB 6.2 MB/s 
     |████████████████████████████████| 895 kB 57.5 MB/s 
     |████████████████████████████████| 636 kB 67.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import stopwordsiso as stopwords
import unicodedata as unicode

In [ ]:
#%tensorflow_version 2.x
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

import os
import errno
import io
tf.__version__

from os import listdir
from os.path import isfile, join, isdir

In [ ]:
import nlpaug.augmenter.word as naw

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('drive')

Mounted at drive


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/augmented_data/train.csv')

In [ ]:
test  = pd.read_csv('https://raw.githubusercontent.com/tblock/10kGNAD/master/test.csv', sep=';', names=['sub','text_1','text_2'])
#train = pd.read_csv('https://raw.githubusercontent.com/tblock/10kGNAD/master/train.csv', sep=';', names=['sub','text'])

In [ ]:
test.fillna('', inplace=True)
train.fillna('', inplace=True)

In [ ]:
test['text'] = test['text_1'] + test['text_2']

In [ ]:
test.drop(['text_1','text_2'],inplace=True, axis=1)

In [ ]:
test.head()

,sub,text
0,Wirtschaft,"'Die Gewerkschaft GPA-djp lanciert den ""All-in..."
1,Sport,Franzosen verteidigen 2:1-Führung – Kritische ...
2,Web,'Neues Video von Designern macht im Netz die R...
3,Sport,23-jähriger Brasilianer muss vier Spiele pausi...
4,International,Aufständische verwendeten Chemikalie bei Gefec...


In [ ]:
text_type_train_list = train['sub'].unique()
text_type_test_list = test['sub'].unique()

In [ ]:
count_files_in_folder = {}
inverse_count_files_in_folder = {}
for text_type in text_type_train_list:
    count_text_type = len(train.loc[train['sub'] == text_type])
    
    count_files_in_folder[text_type] = count_text_type
    inverse_count_files_in_folder[count_text_type] = text_type

In [ ]:
all_values = count_files_in_folder.values()
max_value = max(all_values)
max_value

1510

In [ ]:
#del count_files_in_folder['Sport']
#del count_files_in_folder['Kultur']
#del count_files_in_folder['Wirtschaft']
#del count_files_in_folder['Web']
#del count_files_in_folder['Etat']
#del count_files_in_folder['Inland']


In [ ]:
del count_files_in_folder[inverse_count_files_in_folder[max_value]]

In [ ]:
### get diff to maxvalue for subjects
for key in count_files_in_folder:
    count_files_in_folder[key] = max_value - count_files_in_folder[key]

In [ ]:
count_files_in_folder

{'Etat': 1147,
 'Inland': 1510,
 'International': 1510,
 'Kultur': 1044,
 'Panorama': 1510,
 'Sport': 1510,
 'Web': 1510,
 'Wirtschaft': 1510,
 'Wissenschaft': 1032}

## **Backtranslation**

In [ ]:
def generate_back_translation(df,subject_list=[],subject_to_generate_list=[], sub_column_name='sub', text_column_name='text'):
    for i, subject in enumerate(subject_list):
        amount_to_generate = subject_to_generate_list[i]
        sub_df             = df.loc[df[sub_column_name] == subject]
        print(subject)
        if(i>0):
          with open('/content/drive/MyDrive/Colab Notebooks/augmented_data/'+subject_list[i-1]+".csv", 'w') as f:
            df.to_csv(f, sep=',', index=False) 
        
        for index, row in sub_df.iloc[0:amount_to_generate].iterrows():
            current_text = row['text']
            text_aug     = back_translation_aug.augment(current_text)
            new_row = {sub_column_name:subject, text_column_name:text_aug}
            df = df.append(new_row, ignore_index=True)
            # Write the DataFrame to CSV file.
            with open('/content/drive/MyDrive/Colab Notebooks/augmented_data/train.csv', 'w') as f:
              df.to_csv(f, sep=',', index=False) 

    
    return df

In [ ]:
counts = list(count_files_in_folder.values())
subs   = list(count_files_in_folder.keys())

In [ ]:
back_translation_aug = naw.BackTranslationAug(
    from_model_name='facebook/wmt19-de-en',
    to_model_name='facebook/wmt19-en-de',
    #device='cuda'
)

Downloading:   0%|          | 0.00/825 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/825 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/308k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/829k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/829k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/308k [00:00<?, ?B/s]

In [ ]:
return_df = generate_back_translation(train,subs,counts)

International


Token indices sequence length is longer than the specified maximum sequence length for this model (1319 > 1024). Running this sequence through the model will result in indexing errors


Wissenschaft


In [ ]:
return_df.to_csv('backtranslated_train.csv', sep=';')

In [ ]:
train  = pd.read_csv('backtranslated_train.csv', sep=';', names=['sub','text'])